# ROC curve 

In this section you will learn more about **ROC curves** taught to you thus far in the course. 

## Table of Contents
  
- [1. Introduction](#Intro)  
- [2. Import Packages and Functions](#Import)
- [3. Interpretting the Data](#Inter-data)
- [4. Splitting the Data](#Splitting-data)
- [5. Visualizing a Threshold](#Visual-Thresh)
- [6. Setting a Threshold](#Set-Thresh)
    - [6.1. Challenge #1](#challenge1)
        - [6.1.1. Challenge #1 Example](#challengeEx1)
        - [6.1.2. Challenge #1.1](#challenge1_1)
        - [6.1.3. Challenge #1.2](#challenge1_2)
        - [6.1.4. Challenge #1.3](#challenge1_3)
- [7. ROC Curve](#roc-curve)
- [8. Precision-Recall Curve](#p-r-curve)

<a name = "Intro"></a>
## Introduction

The **ROC Curve** is one of the most useful tools for evaluating a model, particularly in AI for Medicine. ROC curves allow us to analyze a model's classification performance as we vary the **threshold**. The threshold is a parameter in the model that distinguishes two or more different classes. Depending on what the threshold is set to, we can adjust how sensitive our model is in distinguishing certain diseases. This can be incredibly useful for automating the process of classifying diseases. In this lab, you will understand what a threshold is, what it looks like when visualizing one, and seeing how a threshold can generically classify data into classes without overfitting or underfitting the data. We will also take results from a provided threshold and plot an ROC curve in order to distinguish our model's performance as our threshold changes. Finally you will also take a look at a **Precision-Recall Curve** and see how this curve compares with an ROC curve, and when to use one or the other.

<a name="Import"></a>
## Import Packages and Functions

We'll make use of the following packages:
- `numpy` is what we'll use to manipulate our data
- `matplotlib.pyplot` will be used to produce plots for visualization
- `scikit-learn` will be used to support machine learning as well as assist in creating visual models of the data

Run the next cell to import all the necessary packages.

In [ ]:
#import numpy and matplot libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random as rand
import seaborn as sns
import helperROC

In [ ]:
# import sklearn tools from library
from ipywidgets import*
from IPython.display import display
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import precision_recall_curve
from scipy import interp
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

<a name="Inter-data"></a>
## Interpretting the Data

In this section we will organize the data and feed the data into the model. We will be using the **Breast Cancer Wisconsin Data Set**. <a href="https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)" > 
    
More information on this dataset can be found here. </a>
    
We also will split the data into training and test sets. Note that for the data, there are 30 dimensions, but we will only process the first 2 dimensions in order to visualize our graphs earlier.

Since the data has many different features that are difficult to visualize (on a graph, shown below in **Visualizing a Threshold**), you may select which dimensions (1st, 2nd ... 30th) to view the data with. Use the **input** below to input which dimensions you would like to use.

Note: An **input** is a function that allows users to input data, such as text, int, or other data type. More information on the input function can be found <a href="https://www.w3schools.com/python/ref_func_input.asp" > here. 

In [ ]:
print("Enter the first value between 1 and 30 inclusive")
value1 = int(input("Enter Value 1: "))
if (value1 < 1 or value1 > 30):
    print("Please try again.  Enter a value between 1 and 30 inclusive")
    value1 = int(input("Enter Value 1: "))
else:
    print("First data Column: " + str(value1))
    
print("Enter the second value between 1 and 30 inclusive")
value2 = int(input("Enter Value 2: "))
if(value2 < 1 or value2 > 30):
    print("Please try again. Enter a value between 1 and 30 inclusive")
    value2 = int(input("Enter Value 2: "))
else:
    print("Second data Column: " + str(value2))

In [ ]:
# Assign the data into variables, where x is dimensions of the data, and y is the output
breast_cancer = datasets.load_breast_cancer()
X, Y, binClasses = helperROC.inputData(value1, value2, breast_cancer)
# just focusing on selected two dimensions 
#X = breast_cancer.data[:, [value1 - 1, value2 - 1]]
#Y = breast_cancer.target_names
#binClasses = breast_cancer.target

print("Sample data dimensions (Input): ")
print(str(X[:5]))
print("Output: " + str(Y[:5]))
print()
print("Num examples: " + str(X.shape[0]) + " (input)")
print("Selected Dimensions: Columns " + str(value1) + " and " + str(value2))
print("Num labels: "+ str(Y.shape[0]) + " (output)")
print("Unique binary labels: " + str(np.unique(binClasses)))

Above is some sample data from our dataset. 

The **data** includes 569 examples. The total number of features per example is 30, but we are only using 2 just for visual purposes.

The **target** includes 569 labels that determine whether the breast cancer in the data is malignant or benign. We only have 1 label for each example, meaning that an example cannot have both a 0 and a 1 at the same time. 

The above is some sample data, which contains input of 30 dimensions, and an output that classifies as either a 0 or 1. The model will predict based on the patterns of numbers whether another piece of data, containing 30 dimensions, would classify into class 0 or 1. 

<a name="Splitting-data"> </a> 
## Splitting the Data

We are going to manually split our data into training and test sets. This way, we don't have to add in more data and we can assess how our model performs with training on a certain amount of data. 

We have to use ravel() on the trainY because when we split our data, the trainY was 2 dimension. Our data (x) needs to be 2 dimensions because we have 2 different inputs that contribute towards a label (y). However, splitting the dataset caused our label section (or y) to match the shape of our x's, or data. We can only have 1 label per 2 dimension, but having a 2 dimensional array now can cause the data to be misread. Thus, we must use ravel() on trainY to flatten our array back down to 1 dimension, thus matching 2 input dimesnions to 1 output label.

In [ ]:
# split the data into a training and test set
trainX, testX, trainY, testY = train_test_split(X, binClasses, test_size=0.3, random_state=0)

#ravel() allows the outputs to match the shape of a 1 dimensional array. Splitting the data causes the trainY and 
#testY to become a 2 dimensional array, which makes our data very difficult to read. 
trainY = trainY.ravel()
testY = testY.ravel()

In [ ]:
# Define the classes that we are aiming for, which from our data, is 0 and 1
n_classes = np.unique(binClasses).size
random_state = np.random.RandomState(0)
n_samples, n_features = X.shape
print(X.shape)

Now that we have made a training and test dataset, let's verify that we have split the data so that 30% of the data is in the test dataset.

In [ ]:
# Shows how much data we have in each set, and verify the number of dimensions for input
print(f"The training dataset consist of {trainX.shape[0]} samples and {n_features} dimensions. There are {trainY.shape[0]} output labels")
print(f"The testing dataset consist of {testX.shape[0]} samples and {n_features} dimensions. There are {testY.shape[0]} output labels")

## Training the Model

In this section we will train the model with the data that we have from above to classify the data either in class "malignant" or "benign" (0 or 1).

We will use a Logistic Regression model to determine if our patients have malignant or benign breast cancer. Logistic Regression models will determine if our patients have breast cancer or not by using an activation function to yield a percentage between 0 and 1. 0 will represent a negative value, or malignant cancer, and a 1 represents a positive value, or benign breast cancer.

In [ ]:
# Create a classifier that uses Logistic Regression that fits to the training set 
classifier = LogisticRegression(C=1e5)
classFit = classifier.fit(trainX, trainY.ravel())
y_score = classFit.decision_function(testX)
predictions = classifier.predict(testX)

# print some sample predictions and compare it to the output data
print("Sample Predictions: ")
print(str(predictions[:10]))
print("Sample Test Output: ")
print(testY[:10])

<a name="Visual-Thresh"></a>
## Visualizing a threshold

Here is a plot of all of the training data. We can see that the two input dimensions are plotted on the x and y axis, as well as the color to resemble whether or not the patient has breast cancer (0 means healthy, 1 means breast cancer). A **threshold** is a parameter of the model that distinguishes sections of data to define classes in the dataset. The threshold represents how the model will make a prediction on an example based on its parameters. The graph below is an interactive method of showing how a model can generalize how it predicts whether or not exmaples contain breast cancer or not. 

Use the graph below to plot a threshold graph containing the training dataset examples. Fit as many of the blue points in the blue region and red points in the red region. Keep in mind, however, that we do not want to overfit our data. 

In [ ]:
sns.set_style('darkgrid')
hue_colors = {0: 'red', 1: 'blue'}

df = pd.DataFrame(trainX, columns=["feature1", "feature2"])
df["label"] = pd.Series(trainY).apply(lambda x: "red" if x == 1 else "blue")

# account for different cushion values
cushionX = (sum(trainX[value1]) / trainX[value1].size)
cushionY = (sum(trainX[value2]) / trainX[value2].size)
maxY = max(trainX[value2])
maxX = max(trainX[value1])
minY = min(trainX[value2])
minX = min(trainX[value1])

In [ ]:
# test with different dimension combos
def f(slope, yIntercept, xIntercept, degree):
    plt.figure()
    xPlot = np.linspace(minX - cushionX, maxX + cushionX)
    yPlot = slope * ((xPlot + xIntercept) ** degree) + yIntercept

    sns.scatterplot(x="feature1",
                    y="feature2",
                    data=df,
                    hue=trainY,
                    palette=hue_colors)
    if slope < 0:
        plt.fill_between(xPlot, -20, yPlot, facecolor='blue', alpha=0.3)
        plt.fill_between(xPlot, yPlot, 100, facecolor='red', alpha=0.3)
    else:
        plt.fill_between(xPlot, -20, yPlot, facecolor='red', alpha=0.3)
        plt.fill_between(xPlot, yPlot, 100, facecolor='blue', alpha=0.3)
        
    plt.plot(xPlot, yPlot)
    plt.ylim(minY - cushionY, maxY + cushionY)
    plt.xlim(minX - cushionX, maxX + cushionX)

In [ ]:
# fix scaling for each dimension combo
interactive_plot = interactive(f, slope=(-5.00, 5.00), yIntercept=(minY - cushionY, maxY + cushionY, 0.01), 
                                   xIntercept=(minX - cushionX, maxX + cushionX, 0.01), degree=(-2, 4), trainX=trainX, continuous_update=False)
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

<a name="Set-Thresh"></a>
## Setting a Threshold

Now that you know visually what a threshold looks like, let's see what setting a numerical threshold does to the data. 

In [ ]:
# Print sample y_scores, calculated from the Logistic Regression Model and classifier
print("Sample oiginal scores: " + str(y_score[:7]))

We will now create a threshold value that will define the two different classes in our problem, which is 'malignant' and 'benign'. By using the <a href="https://numpy.org/doc/stable/reference/generated/numpy.where.html" > numpy.where </a> function, we can assign particular scores into different classes according to what the threshold is set to.  

In [ ]:
# Define the threshold in order to distinguish classes
threshold = 0.0
# np.where assigns a score calculated from our model into our defined classes
newScores = np.where(y_score < threshold, 0, 1)
newLabels = np.where(y_score < threshold, 'malignant', 'benign')
print("Sample threshold values: " + str(newScores[:7]))
print("Sample threshold values: " + str(newLabels[:7]))

Notice that as we had defined before, the negative, or 0 class represents a "benign" breast cancer, while the positive, or 1 class represents a "malignant" breast cancer. 

<a name="challenge1"></a>
## CHALLENGE

Complete the challenge of finding a value for the threshold. You will be provided an array of scores, and its desired classifications. Use the following sets of data and resulting classification to determine a threshold value that allows for the data to be classified into the result. Enter your answer into the "None" section. Do not change any surrounding code blocks to the threshold answer. 

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<p>
<ul>
    <li> Observe each of the values in the sample. What values should the threshold be in between?</li>
    <li> You will notice that in the threshold may be a value in "region" between a "lower" and "upper" boundary. You only need to enter into the threshold value one single value (integer or float). For instance, if the threshold can be between 2 and 3 without inclusion, entering either 2.2 or 2.5 or any other value in between 2 and 3. You don't need to enter the entire region or values. </li>
    <li> Think about the "lower" and "upper" boundaries of the threshold values. Are the "lower" and "upper" boundaries inclusive or exclusive to fit the desired classification?</li>
    

</ul>
<p>

<a name="challengeEx1"></a>
#### Example

In [ ]:
exampleYScore = helperROC.showChallengeEx()

In [ ]:
threshold = 2

In [ ]:
helperROC.checkAnswerEx(threshold, exampleYScore)

<a name="challenge1_1"></a>
#### Challenge 1

In [ ]:
challenge1_1Score = helperROC.showChallenge1_1()

In [ ]:
threshold = None

In [ ]:
helperROC.checkAnswer1_1(threshold, challenge1_1Score)

<a name="challenge1_2"></a>
#### Challenge 2

In [ ]:
challenge1_2Score = helperROC.showChallenge1_2()

In [ ]:
threshold = None

In [ ]:
helperROC.checkAnswer1_2(threshold, challenge1_2Score)

<a name="challenge1_3"></a>
#### Challenge 3

In [ ]:
challenge1_3Score = helperROC.showChallenge1_3()

In [ ]:
threshold = None

In [ ]:
helperROC.checkAnswer1_3(threshold, challenge1_3Score)

<a name="roc-curve"></a>
## ROC Curve

We will now plot our data onto a **Receiver Operating Characteristic** curve, or **ROC** curve. The ROC curve is a probabilistic curve that determines how likely the model is to classify a piece of data in the correct class. Observe the following image:

<img src="ROC_curves.svg.png" style="width:500px;height:400px"/>

image url: https://www.google.com/url?sa=i&url=https%3A%2F%2Fwww.kdnuggets.com%2F2020%2F03%2Finterpretable-performance-measure-binary-classifier.html&psig=AOvVaw2qE7XoJkQIZXTULBE3289c&ust=1596663161641000&source=images&cd=vfe&ved=0CAIQjRxqFwoTCLiQsZfCgusCFQAAAAAdAAAAABAD

We can clearly observe a **threshold variation** (top left), **confusion matrix** (top right), and **ROC curve** (bottom middle). Notice how each subimage relates to one another. We can clearly see that the confusion matrix distribution can be found graphically with the threshold. Moving the threshold either to the left or the right will modify how many of each category (True/False Positives/Negatives) we have. Also notice that when we move the threshold, we effect the location of the point along the ROC curve. This curve will show us how our model performs because of the number of True and False Positives we achieve. 

In order to calculate the ROC curve, we need to plot the True Positive Rate against False Positive Rate. What we will see is that as the model is testing more data, the model will likely classify more examples correctly, either as True (1) or False (0). Based on how the model was trained, we can analyze the performance of the model with the ROC and the **AUC**, or the **Area Under Curve**. Both the True/False Positive Rates are in between 0 and 1 (where 0 is worst rate, 1 is best rate), as well as the AUC is between 0 and 1 (where 0 is worst performance, 1 is best performance). 

In [ ]:
# calculate the true and false positive rates, which will help us visualize our ROC curve
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(testY, y_score)
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(testY.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
# Plot the ROC curve on the graph, showing the relationship between TP and FP
plt.figure()
lw = 2
plt.plot(fpr[0], tpr[0], color='green',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[0])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

We can see that both the False Positive Rate and True Positive Rate increased with more testing examples. Thus, the area under the curve was also increasing. 

Let's see what the ROC curve looks like with a bad example. Instead of using the Model's predictions, we will just use a random number generator, which will make the overall performance become much less. 

In [ ]:
# Generating and printing a bad example
bad_example = np.random.randint(0, 2, len(testY))
print("Bad test predictions: ")
print(bad_example)

In [ ]:
# Calculate the true and false positive rates, which will help us visualize our ROC curve
fpr2 = dict()
tpr2 = dict()
roc_auc2 = dict()

for i in range(n_classes):
    fpr2[i], tpr2[i], _ = roc_curve(bad_example, y_score)
    roc_auc2[i] = auc(fpr2[i], tpr2[i])

# Compute micro-average ROC curve and ROC area
fpr2["micro"], tpr2["micro"], _ = roc_curve(bad_example, y_score.ravel())
roc_auc2["micro"] = auc(fpr2["micro"], tpr2["micro"])

We can see here that from this graph, the True Positive Rate and False Positive Rate are not nearly as strong as the true model's predictions. The performance from this graph is around 0.5, which is substantially worse than the model's performance, which is around 0.7. We can see that a curve that curves upwards with a higher positive rate will show an overall performance of the model. 

In [ ]:
# Plot the ROC curve on the graph, showing the relationship between TP and FP
plt.figure()
lw = 2
plt.plot(fpr2[0], tpr2[0], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc2[0])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

Let's compare the two ROC curves. How do the two areas compare?

In [ ]:
# Plot the ROC curve on the graph, showing the relationship between TP and FP
plt.figure()
lw = 2
plt.plot(fpr[0], tpr[0], color='green',
         lw=lw, label="Model's ROC curve (area = %0.2f)" % roc_auc[0])
plt.plot(fpr2[0], tpr2[0], color='darkorange',
         lw=lw, label="Random ROC curve (area = %0.2f)" % roc_auc2[0])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

<a name="p-r-curve"></a>
## Precision-Recall Curve

**Precision** is defined as the metrics that determines a model's ability to classify positive examples as positive. Precision is the fraction of the number of true positives divided by true positives plus false positives. This is also very similar to the **Positive Point Value (PPV)** metric.

$$ Precision = PPV = \frac{\text{True Positive}}{\text{True Positive + False Positive}} $$

**Recall** is defined as the model's ability to find all positive examples. the fraction of the number of true positives divided by true positives plus false negatives. This is very similar to the **Sensitivity** metric.

$$ Recall = Sensitivity = \frac{\text{True Positive}}{\text{True Positive + False Negatives}} $$

A **Precision-Recall Curve** is a respresentation that shows the relationship between the precision and recall of a model. In Medcial AI, precision is a measure of results showing a disease, while recall is a measure of how many truly sick patients or patients containing the disease are returned.

You may notice from the definitions of Precision and Recall that we are only focusing on Positive examples, rather than both Positive AND Negative examples. This is because we tend to use Precision-Recall curves when we have an **imbalance** of data, or different distributions of Positive and Negative examples. An ROC curve is not as effective as a Precision-Recall curve because there could be a weighted average in favor of either Positive or Negative examples in an ROC curve. Precision-Recall takes the imbalance into consideration, however can be more difficult to interpret the model's overall performance like the ROC curve can. To review, we could possibly use **data augmentation** to produce more training examples for our model during the training phase. However, if we needed to produce metrics without data augmentation, then a Precision-Recall curve would be useful.  

Below is a plotted Precision-Recall Curve from the model's performance from the data, and the dimensions that you picked at the beginning of this lab. Notice the shape of this curve. How does it compare to an ROC curve? How does the area under the curve (AUC) compare to that of the ROC curve?

In [ ]:
# predict probabilities
lr_probs = classifier.predict_proba(testX)
# keep probabilities for the positive outcome only
lr_probs = lr_probs[:, 1]
# predict class values
yhat = classifier.predict(testX)
lr_precision, lr_recall, _ = precision_recall_curve(testY, lr_probs)
lr_f1, lr_auc = f1_score(testY, yhat), auc(lr_recall, lr_precision)
# summarize scores
print('Logistic: f1=%.3f auc=%.3f' % (lr_f1, lr_auc))
# plot the precision-recall curves
no_skill = len(testY[testY==1]) / len(testY)
plt.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
plt.plot(lr_recall, lr_precision, marker='.', label='Logistic')
# axis labels
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
# show the legend
plt.legend()
# show the plot
plt.show()

Note: 

f1 refers to **F1-score**, which is the **average** between the precision and recall. 

auc refers to **Area Under Curve**, which is the area under the curve, as described above. 

You will notice that an ROC curve and Precision-Recall curve are very similar, both in their shape and representation. Both curves produce a metric that shows the overall performance of the model's classifications through the **Area Under the Curve** (note, the curve itself does not show the model's performance, but rather the area under the curve itself does). The primary difference between these two curves is when to use each curve. We typically use an **ROC curve** when our data distribution between classes is mostly even, whereas if we have a substantial imbalance in the data, we use a **Precision-Recall Curve**.

## Great job!

You now know how to plot and view an ROC curve, and iterpret the contents of such a plot. For instance, you now know that a greater AUC means that the model performed well, which is dependent on the True and False Positive Rates. You will use the information to understand how your model performed with your data, and determine if there are any improvements that you can make to your model. 